<h1>7. Regression</h1>

In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('../student-por.csv', sep=';')
df['finalResult'] = df["G3"].apply(lambda x: 'pass' if x> 9 else 'fail')
df['academicGrade'] = df["G3"].apply(lambda x: 'A' if x>15 and x<21 else 'B' if x>13 and x<16 else 'C' if x>11 and x<14 else 'D' if x>9 and x<12 else 'E' )

<h3>7.1 Preprocess the data. Use the function « ColumnTransformer » to do that. Explain precisely each step, in particular when you drop, impute or transform the variables. Try several approachs.</h3>

 *il y a-t-il des valeurs manquants ?*

In [15]:
df.isnull().any()

school           False
sex              False
age              False
address          False
famsize          False
Pstatus          False
Medu             False
Fedu             False
Mjob             False
Fjob             False
reason           False
guardian         False
traveltime       False
studytime        False
failures         False
schoolsup        False
famsup           False
paid             False
activities       False
nursery          False
higher           False
internet         False
romantic         False
famrel           False
freetime         False
goout            False
Dalc             False
Walc             False
health           False
absences         False
G1               False
G2               False
G3               False
finalResult      False
academicGrade    False
dtype: bool

 *  *identifier les valeur quantitatifs et qualitatifs*

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   school         649 non-null    object
 1   sex            649 non-null    object
 2   age            649 non-null    int64 
 3   address        649 non-null    object
 4   famsize        649 non-null    object
 5   Pstatus        649 non-null    object
 6   Medu           649 non-null    int64 
 7   Fedu           649 non-null    int64 
 8   Mjob           649 non-null    object
 9   Fjob           649 non-null    object
 10  reason         649 non-null    object
 11  guardian       649 non-null    object
 12  traveltime     649 non-null    int64 
 13  studytime      649 non-null    int64 
 14  failures       649 non-null    int64 
 15  schoolsup      649 non-null    object
 16  famsup         649 non-null    object
 17  paid           649 non-null    object
 18  activities     649 non-null   

**Using as predictors the first 32 variables.**

In [17]:
from sklearn.model_selection import train_test_split

data_y = df['G3']
data_x = df.drop(['finalResult','G3','academicGrade',"address","famsize","Pstatus","Mjob","Fjob","reason","guardian","schoolsup"], axis = 1)

x_train,x_test,y_train,y_test = train_test_split(data_x,data_y,test_size=0.20,random_state=1)

data_x.head()

,school,sex,age,Medu,Fedu,traveltime,studytime,failures,famsup,paid,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,GP,F,18,4,4,2,2,0,no,no,...,no,4,3,4,1,1,3,4,0,11
1,GP,F,17,1,1,1,2,0,yes,no,...,no,5,3,3,1,1,3,2,9,11
2,GP,F,15,1,1,1,2,0,no,no,...,no,4,3,2,2,3,3,6,12,13
3,GP,F,15,4,2,1,3,0,yes,no,...,yes,3,2,2,1,1,5,0,14,14
4,GP,F,16,3,3,1,2,0,yes,no,...,no,4,3,2,1,2,5,0,11,13


**Preprocessing**

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

qualitative_var = ["school","sex","famsup","paid","activities","nursery","higher","internet","romantic"]
quantitative_var = ['age','Medu','Fedu','traveltime','studytime','failures','famrel','freetime','goout','Dalc','Walc','health','absences','G1','G2']

#vu que academicGrade et finalResult sont issus de G3, cela n'a pas de sens de les avoir dans le Model

qualitative_pipeline = Pipeline([
     ('encoder', OneHotEncoder(drop = 'first'))
])

quantitative_pipeline = Pipeline([
     ('min_max_scaler', MinMaxScaler())
])
 
preprocessor = ColumnTransformer(remainder='passthrough',
                                 transformers=[
                                   ('transform_quantitatif',qualitative_pipeline, qualitative_var),
                                    ('transform_qualitatif',quantitative_pipeline,quantitative_var),
                                 ])

In [19]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
 
metric_results = {}
def metrics( model, name, result_dic ):
    
    result = model.predict(x_test)
    
    r2 = r2_score(y_test,result)
    mse = mean_squared_error(y_test,result)
    
     
    print(name +' R2 Score :',r2)
    result_dic[name +' R2 Score ' ] = r2
    
    print(name +' Mean Squared Error Score :',mse)
    result_dic[name +' Mean Squared Error Score '] = mse


<h3>Linear Regression</h3> 

In [20]:
from sklearn.linear_model import LinearRegression


LR = Pipeline(
    steps=[("preprocess",preprocessor),
           ("LR",LinearRegression())])

hyperLR = {'LR__fit_intercept':[True,False],'LR__normalize':[True,False]}
gridLR = GridSearchCV(LR,hyperLR,cv=5)

gridLR.fit(x_train,y_train)

modelLR = gridLR.best_estimator_
gridLR.best_params_ 

{'LR__fit_intercept': True, 'LR__normalize': False}

 * <h4>R2 Score, Mean Squared Error.</h4>

In [21]:
metrics(modelLR, 'Linear Regression', metric_results)

Linear Regression R2 Score : 0.8598084721770993
Linear Regression Mean Squared Error Score : 1.8829547165084268


<h3>KNeighbor Regressior</h3> 

In [22]:
from sklearn.neighbors import KNeighborsRegressor


KNR = Pipeline(
    steps=[("preprocess",preprocessor),
           ("KNR",KNeighborsRegressor())])

hyperKNR = {'KNR__n_neighbors':[15,20,25],'KNR__weights':['uniform','distance'],'KNR__leaf_size':[30,40,50],'KNR__p':[1,2]}
gridKNR = GridSearchCV(KNR,hyperKNR,cv=5)

gridKNR.fit(x_train,y_train)

modelKNR = gridKNR.best_estimator_
gridKNR.best_params_ 

{'KNR__leaf_size': 30,
 'KNR__n_neighbors': 15,
 'KNR__p': 1,
 'KNR__weights': 'distance'}

 * <h4>R2 Score, Mean Squared Error.</h4>

In [23]:
metrics(modelKNR, 'KNeighbors Regressor', metric_results)

KNeighbors Regressor R2 Score : 0.4237161385722662
KNeighbors Regressor Mean Squared Error Score : 7.740242450983426


<h3>Decision Tree Regressor</h3> 

In [24]:
from sklearn.tree import DecisionTreeRegressor


DTR = Pipeline(
    steps=[("preprocess",preprocessor),
           ("DTR",DecisionTreeRegressor())])

hyperDTR = {'DTR__criterion':['mse','friedman_mse','mae'],'DTR__splitter':['best','random']}
gridDTR = GridSearchCV(DTR,hyperDTR,cv=5)

gridDTR.fit(x_train,y_train)

modelDTR = gridDTR.best_estimator_
gridDTR.best_params_ 

{'DTR__criterion': 'mse', 'DTR__splitter': 'best'}

 * <h4>R2 Score, Mean Squared Error.</h4>

In [25]:
metrics(modelDTR, 'Decision Tree Regressor', metric_results)

Decision Tree Regressor R2 Score : 0.6025358056998358
Decision Tree Regressor Mean Squared Error Score : 5.338461538461538


<h2>Results</h2>

In [26]:
from ipy_table import *

# dictionary
dict = metric_results

# lists
temp = []
dictList = []

# convert the dictionary to a list
for key, value in dict.items():
    temp = [key,value]
    dictList.append(temp)

# create table with make_table
make_table(dictList)

# apply some styles to the table after it is created
set_column_style(0, width='100', bold=True, color='hsla(225, 80%, 94%, 1)')
set_column_style(1, width='100')

# render the table
render()

Linear Regression R2 Score,0.8598
Linear Regression Mean Squared Error Score,1.8830
KNeighbors Regressor R2 Score,0.4237
KNeighbors Regressor Mean Squared Error Score,7.7402
Decision Tree Regressor R2 Score,0.6025
Decision Tree Regressor Mean Squared Error Score,5.3385
